# Decision Tree Regressor

In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer

In [2]:
grd = pd.read_csv("../data/graphene_data_final.csv")

In [4]:
X, Y = grd[['Graphene_percentage', 'FEED', 'RPM', 'DOC']], grd['MRR_gm_per_sec']

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state=1)

In [8]:
dir(tree)

['BaseDecisionTree',
 'DecisionTreeClassifier',
 'DecisionTreeRegressor',
 'ExtraTreeClassifier',
 'ExtraTreeRegressor',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_classes',
 '_criterion',
 '_export',
 '_reingold_tilford',
 '_splitter',
 '_tree',
 '_utils',
 'export_graphviz',
 'export_text',
 'plot_tree']

In [37]:
tree.DecisionTreeRegressor().fit(X_train,Y_train).score(X_test,Y_test)

0.8456605882420749

In [34]:
for i in range(1,10):
    tree_0 = tree.DecisionTreeRegressor(max_depth=i).fit(X_train,Y_train)   
    print(str(i)+"  "+str(tree_0.score(X_test,Y_test)))

1  0.34215504461832535
2  0.6584615859318392
3  0.7683696377168678
4  0.8427968886279223
5  0.8294883692112021
6  0.8316129305883024
7  0.8497825924673458
8  0.8456605882420749
9  0.8456605882420749


In [36]:
for i in range(1,20):
    tree_0 = tree.DecisionTreeRegressor(min_samples_split=i+1).fit(X_train,Y_train)   
    print(str(i+1)+"  "+str(tree_0.score(X_test,Y_test)))

2  0.8456605882420749
3  0.8470616635414071
4  0.860390263796653
5  0.8701307304048008
6  0.8473999412257508
7  0.8467823180570306
8  0.8461547716851097
9  0.8383191686380057
10  0.8432851400474273
11  0.8432564135002851
12  0.8464894105095315
13  0.8513055680304795
14  0.8513055680304796
15  0.8513055680304796
16  0.8513055680304795
17  0.8513055680304795
18  0.8534638877180606
19  0.8332526763504086
20  0.8332526763504086


In [56]:
for i in range(1,20):
    tree_0 = tree.DecisionTreeRegressor(min_samples_leaf=i+1).fit(X_train,Y_train)   
    print(str(i+1)+"  "+str(tree_0.score(X_test,Y_test)))

2  0.8629876868642068
3  0.847232833145562
4  0.8522967533572814
5  0.8551022398855788
6  0.8493352290787524
7  0.8496606267024551
8  0.8496606267024551
9  0.8330052587584644
10  0.758129224778583
11  0.758129224778583
12  0.7272398824831834
13  0.7213185183452002
14  0.7213185183452002
15  0.7213185183452002
16  0.7213185183452002
17  0.7213185183452002
18  0.7213185183452002
19  0.5695004993981045
20  0.5695004993981045


In [57]:
best_tree = tree.DecisionTreeRegressor(min_samples_leaf=2,min_samples_split=5,max_depth=7)
best_tree.fit(X_train,Y_train)

DecisionTreeRegressor(max_depth=7, min_samples_leaf=2, min_samples_split=5)

In [59]:
best_tree.score(X_test,Y_test)

0.8701307304048008